# PacMan

## Observações de implementação 

- Mapas são registrados em um arquivo *.txt* e deve formatado corretamente com o mesmo número de caracteres em cada linha, apenas um pacman - posicao inicial (P) e pelo menos um objetivo (X)

- Estado definido por tupla (x, y) - posição no mapa

- Parametros para ajustes devem ser definidos na classe Maze

## Restrições para análise do projeto

- Mapas devem conter corredores de espessura de no máximo uma célula ?
- Não são permitidos movimentos na diagonal, apenas, cima, baixo, esquerda e direita
- Pacman não passa mais de uma vez pela mesma célula (busca deve levar em conta states explorados)


In [1]:
from enum import Enum
import time
import copy
from search import *

class PosType(Enum):
    FREE    = ' '
    COIN    = '.'
    B_COIN  = 'o'
    WALL    = '|'
    GHOST   = 'G'
    PACMAN  = 'P'
    GOAL    = 'X'
    PATH    = '+'

    def valid_inits():
        return [ptype for ptype in PosType if ptype not in [PosType.PATH]]
    
    def valid_pacman_positions():
        return [PosType.FREE, PosType.COIN, PosType.B_COIN, PosType.GOAL, PosType.PACMAN]
    
class Maze():
    
    def __init__(self, filepath, free_cost=2, coin_cost=1, bcoin_cost=-10):
        file = open("maze.txt", "r")
        self._maze = file.read().splitlines()
        file.close()
        self.w = len(self._maze[0])
        self.h = len(self._maze)
        self.initial_state = None
        self.goal_states = []
        self.free_cost = free_cost
        self.coin_cost = coin_cost
        self.bcoin_cost = bcoin_cost
        
        for y in range(self.h):
            if len(self._maze[y]) is not self.w:
                raise ValueError('Invalid map dimensions')
            else:
                for x in range(self.w):
                    pos = (x, y)
                    if self.get(pos) is PosType.PACMAN:
                        if self.initial_state is None:
                            self.initial_state = pos
                        else:
                            raise ValueError('Multiple pacmans position')
                    elif self.get(pos) is PosType.GOAL:
                        self.goal_states.append(pos)
                    elif self.get(pos) not in PosType.valid_inits():
                        raise ValueError('Invalid char {}'.format(self._maze[y][x]))
    
    def get(self, pos):
        return PosType(self._maze[pos[1]][pos[0]])
    
    def get_cost(self, pos):
        ptype = self.get(pos)
        if ptype == PosType.COIN:
            return self.coin_cost
        elif ptype == PosType.B_COIN:
            return self.bcoin_cost
        else:
            return self.free_cost
            
    def possible_positions(self, pos):
        x, y = pos[0], pos[1]
        possible_positions = []
        for step in [(0, -1), (1, 0), (0, 1), (-1, 0)]: #up, right, down, left
            newX = self.w - 1 if x+step[0] < 0 else (x+step[0])%self.w
            newY = self.h - 1 if y+step[1] < 0 else (y+step[1])%self.h
            pos = (newX, newY)
            if self.get(pos) in PosType.valid_pacman_positions():
                possible_positions.append(pos)
        
        return possible_positions
    
    def _print(self, maze):
        for line in maze:
            print(''.join(line))
        print('\n')
    
    def __str__(self):
        string = ''
        for line in self._maze:
            string = string+line+'\n'
        return string
        
    def print_solution(self, solution):
        maze = list()
        for line in self._maze:
            maze.append(list(line))
        
        for state in solution:
            maze[state[1]][state[0]] = PosType.PATH.value
        self._print(maze)


class MazePacmanProblem(Problem):

    def __init__(self, initial, goal, maze):
        Problem.__init__(self, initial, goal)
        self.maze = maze

    def actions(self, state):
        return self.maze.possible_positions(state)

    def result(self, state, action):
        return action
    
    def goal_test(self, state):
        if isinstance(self.goal, list):
            return state in self.goal
        else:
            return state == self.goal
    
    def path_cost(self, cost_so_far, stateA, action, stateB):
        return cost_so_far + self.maze.get_cost(action)
        
class Statistics():

    def __init__(self, iterations = 0, expanded = 1, memory = 1):
        self.iterations = iterations
        self.expanded = expanded
        self.memory = memory
        self.time = 0
        self._start_time = time.time()
        
    def update_iterations(self, increment):
        self.iterations = self.iterations + increment
    
    def update_expanded(self, increment):
        self.expanded = self.expanded + increment
        
    def update_memory(self, new_memory):
        if new_memory > self.memory:
            self.memory = new_memory
            
    def finish(self, path_cost):
        self.path_cost = path_cost
        self.time = time.time() - self._start_time
        
    def __str__(self):
        sb = []
        for key in self.__dict__:
            sb.append("{key}='{value}'".format(key=key, value=self.__dict__[key]))

        return ', '.join(sb)

    def __repr__(self):
        return self.__str__() 

# BFS e DFS

In [3]:
def depth_first_search(problem):
    
    statistics = Statistics()
    
    frontier = [(Node(problem.initial))] 

    explored = set()
    while frontier:
        statistics.update_memory(len(frontier))
        statistics.update_iterations(1)
        node = frontier.pop()
        if problem.goal_test(node.state):
            statistics.finish(node.path_cost)
            return statistics, node
        explored.add(node.state)
        childs = [child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier]
        frontier.extend(childs)
        statistics.update_expanded(len(childs))
                                 
    statistics.finish_time()
    return statistics, None


def breadth_first_search(problem):
    statistics = Statistics()
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = deque([node])
    explored = set()
    while frontier:
        statistics.update_memory(len(frontier))
        statistics.update_iterations(1)
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    statistics.finish(node.path_cost)
                    return statistics, child
                frontier.append(child)
                statistics.update_expanded(1)
    statistics.finish_time()
    return statistics, None

In [4]:
maze = Maze("maze.txt")
problem = MazePacmanProblem(maze.initial_state, maze.goal_states, maze)
print(maze)
statistics, node = breadth_first_search(problem)
print(statistics)
maze.print_solution(node.solution())

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ....|||||||
|||||||.||X||||||.||.|||||||
|....||.||.||||||.||.||....|
|.||.||.||........||.||.||.|
|.||....||||||||||||....||.|
|.||.||.||||||||||||.||.||.|
|....||..............||...P|
||||||||||||||||||||||||||||

iterations='62', expanded='68', memory='6', time='0.0025336742401123047', _start_time='1588018857.79

In [6]:
maze = Maze("maze.txt")
problem = MazePacmanProblem(maze.initial_state, maze.goal_states, maze)
print(maze)
statistics, node = depth_first_search(problem)
print(statistics)
maze.print_solution(node.solution())

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ....|||||||
|||||||.||X||||||.||.|||||||
|....||.||.||||||.||.||....|
|.||.||.||........||.||.||.|
|.||....||||||||||||....||.|
|.||.||.||||||||||||.||.||.|
|....||..............||...P|
||||||||||||||||||||||||||||

iterations='95', expanded='104', memory='10', time='0.005239963531494141', _start_time='1588018886.3

In [7]:
def best_first_search(problem, f, display=False):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None


def uniform_cost_search(problem, display=False):
    return best_first_search(problem, lambda node: node.path_cost, display)

In [8]:
maze = Maze("maze.txt")
problem = MazePacmanProblem(maze.initial_state, maze.goal_states, maze)
node = uniform_cost_search(problem)
maze.print_solution(node.solution())
print(len(node.solution()))

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ++++|||||||
|||||||.||+||||||+||+|||||||
|....||.||+||||||+||+||....|
|.||.||.||++++++++||+||.||.|
|.||....||||||||||||++++||.|
|.||.||.||||||||||||.||+||.|
|....||..............||+++P|
||||||||||||||||||||||||||||


27


In [9]:
def hill_climbing(problem):
    current = Node(problem.initial)
    while True:
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
            break
        current = neighbor
    return current.state

In [ ]:
maze = Maze("maze.txt")
problem = MazePacmanProblem(maze.initial_state, maze.goal_states, maze)
node = uniform_cost_search(problem)
maze.print_solution(node.solution())
print(len(node.solution()))

# TODO

- Definir funções para plotar grafico com as estatisticas
   
- Documentar codigo

- Analise DFS vs BFS

- Analise Uniform Cost Search

- Analise A* e 2 heuristicas (comparar com uniformed search)

- Analise Hill clibbing

- Definir cenários generico para avaliação dos algoritmos

- Tabela com todos os resultados
    - nome do mapa, algoritmo, utilizado estatisticas
    - agrupar por algoritmo, calculando a média, max, min, desvio padrao das estatisicas

- Graficos dos experimentos
    - Graficos numero de iterações vs distancia para objetivo 
    - Graficos memoria maxima utilizada vs distancia para objetivo

- Gerar imagem do mapa (mais bonito)